In [16]:
import pandas as pd

# Load the dataset
file_path = "train.csv"
df = pd.read_csv(file_path)

# Check for missing values
missing_values = df.isnull().sum()
missing_values[missing_values > 0]  # Displaying only columns with missing values

Series([], dtype: int64)

In [10]:
pip install torch

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.2 -> 24.3.1
[notice] To update, run: C:\Users\rafif\AppData\Local\Programs\Python\Python312\python.exe -m pip install --upgrade pip


In [11]:
import torch
import pandas as pd
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

In [14]:
train_data = TensorDataset(torch.tensor(file_path, dtype=torch.float32), torch.tensor(df_train_y, dtype=torch.float32))
test_data = TensorDataset(torch.tensor(file_path, dtype=torch.float32), torch.tensor(df_test_y, dtype=torch.float32))

class NeuralNetwork(nn.Module):
    def __init__(self, input_size, output_size, depth, width, activation):
        super(NeuralNetwork, self).__init__()
        layers = []
        prev_layer_size = input_size
        for _ in range(depth):
            layers.append(nn.Linear(prev_layer_size, width))
            if activation == 'tanh':
                nn.init.xavier_uniform_(layers[-1].weight)  # Xavier initialization for tanh
            elif activation == 'relu':
                nn.init.kaiming_uniform_(layers[-1].weight, nonlinearity='relu')  # He initialization for ReLU
            layers.append(nn.Tanh() if activation == 'tanh' else nn.ReLU())
            prev_layer_size = width
        layers.append(nn.Linear(prev_layer_size, output_size))
        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)

def train_model(model, train_loader, criterion, optimizer, epochs=10):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels.unsqueeze(1))
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

def evaluate_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs)
            predicted = (outputs >= 0.5).float()
            total += labels.size(0)
            correct += (predicted == labels.unsqueeze(1)).sum().item()
    accuracy = correct / total
    return accuracy

input_size = df_train_X.shape[1]
output_size = 1
batch_size = 32
learning_rate = 1e-3

depths = [3, 5, 9]
widths = [5, 10, 25, 50, 100]
activations = ['tanh', 'relu']

for depth in depths:
    for width in widths:
        for activation in activations:
            model = NeuralNetwork(input_size, output_size, depth, width, activation)
            criterion = nn.BCEWithLogitsLoss()
            optimizer = optim.Adam(model.parameters(), lr=learning_rate)
            train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
            test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

            print(f"Training with depth={depth}, width={width}, activation={activation}")
            train_model(model, train_loader, criterion, optimizer)
            train_accuracy = evaluate_model(model, train_loader)
            test_accuracy = evaluate_model(model, test_loader)
            print(f"Train error: {1 - train_accuracy}")
            print(f"Test error: {1 - test_accuracy}")

TypeError: new(): invalid data type 'str'